In [9]:
import sys
!{sys.executable} -m pip install elasticsearch==6.8.2

     |████████████████████████████████| 90 kB 1.4 MB/s eta 0:00:01


In [10]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [16]:
ELASTIC_URL = '0.0.0.0:9200'

es = Elasticsearch(ELASTIC_URL)
es.info()

{'name': '-orWJ9H',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'wGvvrOBZT12LXKS_LIJATg',
 'version': {'number': '6.5.4',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd2ef93d',
  'build_date': '2018-12-17T21:17:40.758843Z',
  'build_snapshot': False,
  'lucene_version': '7.5.0',
  'minimum_wire_compatibility_version': '5.6.0',
  'minimum_index_compatibility_version': '5.0.0'},
 'tagline': 'You Know, for Search'}

In [30]:
t9_index = {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "lowercase",
              "custom_edge_ngram"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram": 2,
            "max_gram": 10
          }
        }
      }
    }
  },
  "mappings": {
    "my_type": {
      "properties": {
        "text": {
          "type": "text",
          "analyzer": "custom_analyzer",
          "search_analyzer": "standard"
        }
      }
    }
  }
}

In [35]:
es.indices.create('test_t9', body=t9_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_t9'}

In [46]:
print(json.dumps(es.indices.get_settings('test_t9'), indent=4))

{
    "test_t9": {
        "settings": {
            "index": {
                "number_of_shards": "5",
                "provided_name": "test_t9",
                "creation_date": "1615578363995",
                "analysis": {
                    "filter": {
                        "custom_edge_ngram": {
                            "type": "edge_ngram",
                            "min_gram": "2",
                            "max_gram": "10"
                        }
                    },
                    "analyzer": {
                        "custom_analyzer": {
                            "filter": [
                                "lowercase",
                                "custom_edge_ngram"
                            ],
                            "type": "custom",
                            "tokenizer": "standard"
                        }
                    }
                },
                "number_of_replicas": "1",
                "uuid": "Q5bamA9eSgO5NecZjq4jrA"

In [48]:
from typing import Generator, List

import psycopg2


class PostgresStorage:
    """
    Base class for working with PostgreSQL
    """

    conn: psycopg2.extensions.connection

    def __init__(self, conn):
        self.conn = conn

    @classmethod
    def connect(cls,
                host: str,
                port: int = 5432,
                user: str = 'postgres',
                password: str = 'password',
                dbname: str = 'postgres'):
        return cls(conn=psycopg2.connect(
            host=host, port=port, user=user, password=password, dbname=dbname)
        )

    def exec_query(self, query: str, params: List[str]) -> Generator:
        cursor = self.conn.cursor()
        try:
            cursor.execute(query, params)
        except psycopg2.Error as e:
            self.conn.rollback()
            raise e
        return cursor.fetchall()

    def exec(self, sql: str, params: List[str]):
        cursor = self.conn.cursor()
        try:
            cursor.execute(sql, params)
        except psycopg2.Error as e:
            self.conn.rollback()
            raise e
        self.conn.commit()

In [63]:
import os

PG_USER = os.environ.get('PG_USER', 'postgres')
PG_PASS = os.environ.get('PG_PASS', 'password')
PG_HOST = os.environ.get('PG_HOST', '172.17.0.2')
PG_PORT = os.environ.get('PG_PORT', 5432)
PG_DBNAME = os.environ.get('PG_DBNAME', 'vnkrtv')

In [66]:
storage = PostgresStorage.connect(
    host=PG_HOST,
    port=PG_PORT,
    user=PG_USER,
    password=PG_PASS,
    dbname=PG_DBNAME
)

In [88]:
from typing import Generator, Iterable
import re

import tqdm
import nltk
import razdel


class Tokenizer:
    remove_punctuation = re.compile(r'[^а-яА-ЯёЁ ]')

    @classmethod
    def tokenize(cls, text: str) -> Generator:
        return (cls.remove_punctuation.sub('', sentence.text.lower().strip())
                for sentence in razdel.sentenize(text))


class TextProcessor:
    tokenizer = Tokenizer()

    @classmethod
    def get_sentences_gens(cls, texts: Iterable) -> Generator:
        for text in texts:
            for sentence in cls.tokenizer.tokenize(text):
                yield sentence
                
    @classmethod
    def get_word_lists_gen(cls, texts: Iterable) -> Generator:
        for text in tqdm.tqdm(texts):
            for sentence in cls.tokenizer.tokenize(text):
                yield [_.text for _ in razdel.tokenize(sentence)]

    @classmethod
    def get_text_gen(cls, text_gen: Iterable) -> Generator:
        for sentence in cls.get_sentences_gens(text_gen):
            yield [_.text for _ in razdel.tokenize(sentence)]

    @classmethod
    def get_ngram_gen(cls, text_gen: Iterable, ngram_size: int = 3) -> Generator:
        for sentence in cls.get_sentences_gens(text_gen):
            yield [''.join(item) for item in nltk.ngrams(sentence, ngram_size)]


In [70]:
train_text = [
    row[0] for row in storage.exec_query('''
    SELECT text
      FROM posts INNER JOIN habs 
        ON posts.post_id = habs.post_id 
     WHERE hab = %s 
     LIMIT 2000''', params=['Машинное обучение'])
]
len(train_text)

2000

In [96]:
t9_index = {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "lowercase",
              "custom_edge_ngram"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram": 2,
            "max_gram": 10
          }
        }
      }
    }
  },
  "mappings": {
    "sentence": {
      "properties": {
        "text": {
          "type": "text",
          "analyzer": "custom_analyzer",
          "search_analyzer": "standard"
        }
      }
    }
  }
}
es.indices.create('t9_app_3', body=t9_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 't9_app_3'}

In [98]:
window_size = 5

for sentence in TextProcessor.get_word_lists_gen(train_text):
#     for i in range(len(sentence) // window_size):
#         words = sentence[i * window_size: (i + 1) * window_size]
#         text = ' '.join(sentence[i * window_size: (i + 1) * window_size])
#         print(text)
    es.index(index='t9_app_3', doc_type='sentence', body={
        'text': ' '.join(sentence)
    })
#         if len(words) < window_size:
#             break






  0%|          | 0/2000 [00:00<?, ?it/s]




  0%|          | 1/2000 [00:02<1:21:08,  2.44s/it]




  0%|          | 2/2000 [00:03<1:04:12,  1.93s/it]




  0%|          | 3/2000 [00:03<51:44,  1.55s/it]  




  0%|          | 4/2000 [00:04<43:57,  1.32s/it]




  0%|          | 5/2000 [00:05<40:25,  1.22s/it]




  0%|          | 6/2000 [00:06<40:06,  1.21s/it]




  0%|          | 7/2000 [00:07<31:26,  1.06it/s]




  0%|          | 8/2000 [00:07<25:28,  1.30it/s]




  0%|          | 9/2000 [00:08<27:03,  1.23it/s]




  0%|          | 10/2000 [00:08<23:08,  1.43it/s]




  1%|          | 11/2000 [00:09<21:55,  1.51it/s]




  1%|          | 12/2000 [00:13<54:10,  1.63s/it]




  1%|          | 13/2000 [00:14<45:34,  1.38s/it]




  1%|          | 14/2000 [00:14<37:00,  1.12s/it]




  1%|          | 15/2000 [00:15<33:45,  1.02s/it]




  1%|          | 16/2000 [00:16<30:15,  1.09it/s]




  1%|          | 17/2000 [00:16<29:23,  1.12it/s]




  1%|          | 18/2000 [00:17<29:

  6%|▌         | 121/2000 [02:12<1:11:16,  2.28s/it]




  6%|▌         | 122/2000 [02:14<59:59,  1.92s/it]  




  6%|▌         | 123/2000 [02:15<56:05,  1.79s/it]




  6%|▌         | 124/2000 [02:17<58:03,  1.86s/it]




  6%|▋         | 125/2000 [02:19<55:18,  1.77s/it]




  6%|▋         | 126/2000 [02:20<49:08,  1.57s/it]




  6%|▋         | 127/2000 [02:21<43:23,  1.39s/it]




  6%|▋         | 128/2000 [02:23<52:36,  1.69s/it]




  6%|▋         | 129/2000 [02:26<1:06:52,  2.14s/it]




  6%|▋         | 130/2000 [02:27<49:13,  1.58s/it]  




  7%|▋         | 131/2000 [02:28<49:06,  1.58s/it]




  7%|▋         | 132/2000 [02:28<36:32,  1.17s/it]




  7%|▋         | 133/2000 [02:29<32:09,  1.03s/it]




  7%|▋         | 134/2000 [02:30<34:55,  1.12s/it]




  7%|▋         | 135/2000 [02:34<54:59,  1.77s/it]




  7%|▋         | 136/2000 [02:36<58:22,  1.88s/it]




  7%|▋         | 137/2000 [02:37<53:59,  1.74s/it]




  7%|▋         | 138/2000 [02:42<1:26:11,  2.78s/it]






 12%|█▏        | 237/2000 [05:25<36:33,  1.24s/it]




 12%|█▏        | 238/2000 [05:28<50:20,  1.71s/it]




 12%|█▏        | 239/2000 [05:30<54:14,  1.85s/it]




 12%|█▏        | 240/2000 [05:32<55:26,  1.89s/it]




 12%|█▏        | 241/2000 [05:34<55:56,  1.91s/it]




 12%|█▏        | 242/2000 [05:35<47:15,  1.61s/it]




 12%|█▏        | 243/2000 [05:36<42:59,  1.47s/it]




 12%|█▏        | 244/2000 [05:36<34:04,  1.16s/it]




 12%|█▏        | 245/2000 [05:37<27:07,  1.08it/s]




 12%|█▏        | 246/2000 [05:38<31:49,  1.09s/it]




 12%|█▏        | 247/2000 [05:39<30:04,  1.03s/it]




 12%|█▏        | 248/2000 [05:41<36:28,  1.25s/it]




 12%|█▏        | 249/2000 [05:41<27:11,  1.07it/s]




 12%|█▎        | 250/2000 [05:42<29:15,  1.00s/it]




 13%|█▎        | 251/2000 [05:43<31:57,  1.10s/it]




 13%|█▎        | 252/2000 [05:45<32:39,  1.12s/it]




 13%|█▎        | 253/2000 [05:45<29:27,  1.01s/it]




 13%|█▎        | 254/2000 [05:46<29:56,  1.03s/it]




 13%|█▎   

 18%|█▊        | 354/2000 [08:01<43:04,  1.57s/it]




 18%|█▊        | 355/2000 [08:03<48:25,  1.77s/it]




 18%|█▊        | 356/2000 [08:04<43:59,  1.61s/it]




 18%|█▊        | 357/2000 [08:05<35:08,  1.28s/it]




 18%|█▊        | 358/2000 [08:05<30:46,  1.12s/it]




 18%|█▊        | 359/2000 [08:09<51:18,  1.88s/it]




 18%|█▊        | 360/2000 [08:13<1:04:09,  2.35s/it]




 18%|█▊        | 361/2000 [08:15<1:05:45,  2.41s/it]




 18%|█▊        | 362/2000 [08:15<48:29,  1.78s/it]  




 18%|█▊        | 363/2000 [08:16<36:57,  1.35s/it]




 18%|█▊        | 364/2000 [08:17<34:15,  1.26s/it]




 18%|█▊        | 365/2000 [08:18<34:59,  1.28s/it]




 18%|█▊        | 366/2000 [08:18<25:50,  1.05it/s]




 18%|█▊        | 367/2000 [08:20<31:26,  1.16s/it]




 18%|█▊        | 368/2000 [08:21<30:40,  1.13s/it]




 18%|█▊        | 369/2000 [08:22<25:50,  1.05it/s]




 18%|█▊        | 370/2000 [08:30<1:25:16,  3.14s/it]




 19%|█▊        | 371/2000 [08:30<1:00:50,  2.24s/it]






 24%|██▎       | 472/2000 [11:02<36:38,  1.44s/it]




 24%|██▎       | 473/2000 [11:04<35:40,  1.40s/it]




 24%|██▍       | 475/2000 [11:05<31:23,  1.24s/it]




 24%|██▍       | 476/2000 [11:08<41:46,  1.64s/it]




 24%|██▍       | 477/2000 [11:09<36:10,  1.42s/it]




 24%|██▍       | 478/2000 [11:11<44:16,  1.75s/it]




 24%|██▍       | 479/2000 [11:12<36:57,  1.46s/it]




 24%|██▍       | 480/2000 [11:13<35:40,  1.41s/it]




 24%|██▍       | 481/2000 [11:14<29:37,  1.17s/it]




 24%|██▍       | 482/2000 [11:22<1:18:31,  3.10s/it]




 24%|██▍       | 483/2000 [11:23<1:02:34,  2.47s/it]




 24%|██▍       | 484/2000 [11:25<1:03:55,  2.53s/it]




 24%|██▍       | 485/2000 [11:26<52:01,  2.06s/it]  




 24%|██▍       | 486/2000 [11:27<43:37,  1.73s/it]




 24%|██▍       | 487/2000 [11:27<33:04,  1.31s/it]




 24%|██▍       | 488/2000 [11:28<30:12,  1.20s/it]




 24%|██▍       | 489/2000 [11:30<30:53,  1.23s/it]




 24%|██▍       | 490/2000 [11:31<29:35,  1.18s/it]




 2

...

 76%|███████▌  | 1519/2000 [36:45<14:24,  1.80s/it]




 76%|███████▌  | 1520/2000 [36:46<12:26,  1.56s/it]




 76%|███████▌  | 1521/2000 [36:48<13:14,  1.66s/it]




 76%|███████▌  | 1522/2000 [36:53<21:30,  2.70s/it]




 76%|███████▌  | 1523/2000 [36:55<18:50,  2.37s/it]




 76%|███████▌  | 1524/2000 [36:56<16:37,  2.10s/it]




 76%|███████▋  | 1525/2000 [36:57<12:30,  1.58s/it]




 76%|███████▋  | 1526/2000 [36:58<13:05,  1.66s/it]




 76%|███████▋  | 1527/2000 [37:00<13:30,  1.71s/it]




 76%|███████▋  | 1528/2000 [37:01<11:19,  1.44s/it]




 76%|███████▋  | 1529/2000 [37:02<09:52,  1.26s/it]




 76%|███████▋  | 1530/2000 [37:02<07:22,  1.06it/s]




 77%|███████▋  | 1531/2000 [37:03<06:36,  1.18it/s]




 77%|███████▋  | 1532/2000 [37:05<10:04,  1.29s/it]




 77%|███████▋  | 1533/2000 [37:07<11:24,  1.47s/it]




 77%|███████▋  | 1534/2000 [37:09<11:33,  1.49s/it]




 77%|███████▋  | 1535/2000 [37:09<09:21,  1.21s/it]




 77%|███████▋  | 1536/2000 [37:09<07:15,  1.07it

 82%|████████▏ | 1635/2000 [39:45<08:23,  1.38s/it]




 82%|████████▏ | 1636/2000 [39:46<07:24,  1.22s/it]




 82%|████████▏ | 1637/2000 [39:47<07:01,  1.16s/it]




 82%|████████▏ | 1638/2000 [39:50<09:31,  1.58s/it]




 82%|████████▏ | 1639/2000 [39:50<07:11,  1.20s/it]




 82%|████████▏ | 1640/2000 [39:51<06:27,  1.08s/it]




 82%|████████▏ | 1641/2000 [39:51<05:58,  1.00it/s]




 82%|████████▏ | 1642/2000 [39:54<08:04,  1.35s/it]




 82%|████████▏ | 1643/2000 [39:55<08:11,  1.38s/it]




 82%|████████▏ | 1644/2000 [39:56<06:59,  1.18s/it]




 82%|████████▏ | 1645/2000 [39:56<05:25,  1.09it/s]




 82%|████████▏ | 1646/2000 [39:58<06:26,  1.09s/it]




 82%|████████▏ | 1647/2000 [39:59<06:17,  1.07s/it]




 82%|████████▏ | 1648/2000 [40:01<08:59,  1.53s/it]




 82%|████████▏ | 1649/2000 [40:02<07:40,  1.31s/it]




 82%|████████▎ | 1650/2000 [40:02<06:03,  1.04s/it]




 83%|████████▎ | 1651/2000 [40:08<13:48,  2.37s/it]




 83%|████████▎ | 1652/2000 [40:09<11:30,  1.98s/

 88%|████████▊ | 1751/2000 [42:38<07:41,  1.85s/it]




 88%|████████▊ | 1752/2000 [42:41<09:05,  2.20s/it]




 88%|████████▊ | 1753/2000 [42:42<07:00,  1.70s/it]




 88%|████████▊ | 1754/2000 [42:46<09:50,  2.40s/it]




 88%|████████▊ | 1755/2000 [42:47<08:39,  2.12s/it]




 88%|████████▊ | 1756/2000 [42:48<06:44,  1.66s/it]




 88%|████████▊ | 1757/2000 [42:50<06:38,  1.64s/it]




 88%|████████▊ | 1758/2000 [42:50<05:21,  1.33s/it]




 88%|████████▊ | 1759/2000 [42:57<11:27,  2.85s/it]




 88%|████████▊ | 1760/2000 [42:58<09:25,  2.35s/it]




 88%|████████▊ | 1761/2000 [42:59<07:38,  1.92s/it]




 88%|████████▊ | 1762/2000 [43:00<06:42,  1.69s/it]




 88%|████████▊ | 1763/2000 [43:01<05:50,  1.48s/it]




 88%|████████▊ | 1764/2000 [43:03<06:06,  1.55s/it]




 88%|████████▊ | 1765/2000 [43:06<08:01,  2.05s/it]




 88%|████████▊ | 1766/2000 [43:07<06:47,  1.74s/it]




 88%|████████▊ | 1767/2000 [43:08<06:28,  1.67s/it]




 88%|████████▊ | 1768/2000 [43:10<06:01,  1.56s/

 93%|█████████▎| 1867/2000 [45:10<02:18,  1.04s/it]




 93%|█████████▎| 1868/2000 [45:11<01:47,  1.22it/s]




 93%|█████████▎| 1869/2000 [45:11<01:43,  1.27it/s]




 94%|█████████▎| 1870/2000 [45:12<01:43,  1.26it/s]




 94%|█████████▎| 1871/2000 [45:16<03:46,  1.75s/it]




 94%|█████████▎| 1872/2000 [45:17<03:03,  1.43s/it]




 94%|█████████▎| 1873/2000 [45:19<03:22,  1.59s/it]




 94%|█████████▎| 1874/2000 [45:19<02:27,  1.17s/it]




 94%|█████████▍| 1875/2000 [45:20<02:01,  1.03it/s]




 94%|█████████▍| 1876/2000 [45:20<01:43,  1.20it/s]




 94%|█████████▍| 1877/2000 [45:20<01:23,  1.47it/s]




 94%|█████████▍| 1878/2000 [45:24<02:56,  1.45s/it]




 94%|█████████▍| 1879/2000 [45:24<02:14,  1.11s/it]




 94%|█████████▍| 1880/2000 [45:26<02:30,  1.25s/it]




 94%|█████████▍| 1881/2000 [45:27<02:40,  1.35s/it]




 94%|█████████▍| 1882/2000 [45:28<02:05,  1.06s/it]




 94%|█████████▍| 1883/2000 [45:30<02:54,  1.49s/it]




 94%|█████████▍| 1884/2000 [45:32<02:55,  1.51s/

 99%|█████████▉| 1982/2000 [48:10<00:31,  1.75s/it]




 99%|█████████▉| 1983/2000 [48:14<00:41,  2.42s/it]




 99%|█████████▉| 1984/2000 [48:15<00:34,  2.17s/it]




 99%|█████████▉| 1985/2000 [48:17<00:28,  1.92s/it]




 99%|█████████▉| 1986/2000 [48:17<00:22,  1.60s/it]




 99%|█████████▉| 1987/2000 [48:18<00:16,  1.24s/it]




 99%|█████████▉| 1988/2000 [48:19<00:13,  1.11s/it]




 99%|█████████▉| 1989/2000 [48:20<00:14,  1.30s/it]




100%|█████████▉| 1990/2000 [48:22<00:13,  1.35s/it]




100%|█████████▉| 1991/2000 [48:23<00:11,  1.31s/it]




100%|█████████▉| 1992/2000 [48:25<00:11,  1.46s/it]




100%|█████████▉| 1993/2000 [48:26<00:08,  1.25s/it]




100%|█████████▉| 1994/2000 [48:29<00:10,  1.83s/it]




100%|█████████▉| 1995/2000 [48:30<00:07,  1.55s/it]




100%|█████████▉| 1996/2000 [48:32<00:06,  1.68s/it]




100%|█████████▉| 1997/2000 [48:35<00:06,  2.14s/it]




100%|█████████▉| 1998/2000 [48:36<00:03,  1.70s/it]




100%|█████████▉| 1999/2000 [48:38<00:01,  1.81s/

In [74]:
import tqdm

for sentence in tqdm.tqdm(train_corpus):
    es.index(index='test_t9', doc_type='my_type', body={
        'text': sentence
    })

 17%|█▋        | 31900/187405 [08:04<42:24, 61.11it/s]  

KeyboardInterrupt: 